In [115]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from tqdm import tqdm
import xgboost as xgb
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import warnings
import os
import gdown
import zipfile

warnings.filterwarnings("ignore")
one_hot = OneHotEncoder()

In [ ]:
current_path = os.path.realpath("test.ipynb")
url = "https://drive.google.com/uc?id=1Do4O2zH0Gq197n_e-oEdzRYGSEl8W6BX"
path_to_download = current_path[:len(current_path) - 10]

gdown.download(url, path_to_download, quiet=False)

zip_path = os.path.realpath("train.zip")
extract_path = zip_path[:len(zip_path) - 9]

with zipfile.ZipFile(zip_path, "r") as ext:
    ext.extractall(extract_path)

In [3]:
def sumG(records):
    res = np.zeros(records.shape[1])
    for rec in records:
        res += rec

    return res

In [4]:
def findMax(record):
  return max(record), np.where(record == max(record))[0][0]

In [5]:
def findLeftBorder(record, indVal):
  index = indVal
  val = 0

  # if index == 0:
  #   return val, index

  while record[index] > record[index - 1]:
    val = record[index - 1]
    index -= 1

  return val, index

In [27]:
def findRightBorder(record, indVal):
  index = indVal
  val = 0

  # if index + 1 >= record.shape[0]:
  #   return val, index
  while index < 4999 and record[index] > record[index + 1]:
    val = record[index + 1]
    index += 1

  while index < 4999 and record[index] < record[index + 1]:
    val = record[index + 1]
    index += 1

  return val, index

In [7]:
def isDiff(prevMax, currVal):
    return True if abs(currVal - prevMax) < (1e-3) and currVal > 5 else False

In [9]:
def findQRS(recordTr):
    record = recordTr.copy()

    prevMax = max(record)
    maxVal = max(record)
    leftInd, indVal,rightInd = 0, 0, 0

    qrsInd = []

    i = 0

    while isDiff(prevMax, maxVal):
        maxVal, indVal = findMax(record)
    
        leftVal, leftInd = findLeftBorder(record, indVal)
        rightVal, rightInd = findRightBorder(record, indVal)
        # print([[leftInd, leftVal], [indVal, maxVal], [rightInd, rightVal]])
        qrsInd.append([[leftInd, leftVal], [indVal, maxVal], [rightInd, rightVal]])

        prevMax = maxVal
    
        for ind in range(leftInd, rightInd + 1):
            record[ind] = 0

        i += 1

    return qrsInd[:-1]

In [ ]:
df = pd.read_csv("train/train_meta.csv")

files = df["record_name"]

files

0       00009_hr
1       00034_hr
2       00043_hr
3       00052_hr
4       00057_hr
          ...   
2096    21784_hr
2097    21795_hr
2098    21825_hr
2099    21831_hr
2100    21834_hr
Name: record_name, Length: 2101, dtype: object

In [86]:
def count(files, fl=True):

    QRS_count = []
    QRS = []

    path = "train" if fl else "train/test"

    for i, g in tqdm(enumerate(files)):
        recordTr = sumG(np.load(f"{path}/{g}.npy"))
        Index = findQRS(recordTr)

        QRS_count.append(len(Index))
        QRS.append(Index)

    return QRS_count, QRS

In [82]:
df["QRS_count"] = count(files)[0]

2101it [00:10, 191.54it/s]


In [77]:
X, y = one_hot.fit_transform(df).toarray(), pd.read_csv("train/train_gts.csv")["myocard"]

In [91]:
X.shape

(2101, 15157)

In [66]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [67]:
model = xgb.XGBClassifier(enable_categorical=True)

In [68]:
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=True, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [71]:
pred = model.predict(X_test)

acc = accuracy_score(y_test, pred)

print(f"Accuracy is about {acc:.5f}")

Accuracy is about 0.77910
